In [1]:
from transformers import pipelines
import tensorflow as tf
import numpy as np
import os
import datasets
from transformers import RobertaTokenizer
from datasets import load_dataset, concatenate_datasets

/Users/chahal/Desktop/codedetective/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def load_and_preprocess_data(language):
    # Load the dataset
    dataset = load_dataset('code_search_net', language)

    # Combine the training and validation datasets
    combined_dataset = dataset['train'].concatenate(dataset['validation'])

    # Shuffle the combined dataset
    combined_dataset = combined_dataset.shuffle(seed=42)

    # Split the dataset into 75% training and 25% testing
    train_test_split = combined_dataset.train_test_split(test_size=0.25, seed=42)
    train_dataset = train_test_split['train']
    test_dataset = train_test_split['test']

    # Split the training dataset into 90% training and 10% validation
    train_validation_split = train_dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = train_validation_split['train']
    validation_dataset = train_validation_split['test']
    return train_dataset, validation_dataset, test_dataset




In [7]:
#tokenize the datasets return tensorflow tensors
def preprocess_tokenizeation(dataset):
    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base",trust_remote_code=True)
    def preprocess_function(inputs):
        
        return tokenizer(inputs["code"],trunacted = True,  padding='max_length', max_length=512, return_tensors="tf")
    tokenized_data = dataset.map(preprocess_function,batched=True)

    
    return tokenized_data

#function converts tokenized data to a tf dataset for performance
def to_tf(tokenized_data,batch_size=16):
    return tokenized_data.to_tf_dataset(columns=["input_ids", "attention_mask", "label"],
            shuffle=True,
            batch_size=batch_size,
            collate_fn=lambda x: {'input_ids': x['input_ids'], 'attention_mask': x['attention_mask'], 'labels': x['label']}
        )


In [11]:
# calls the above functions for the python and js datasets and then tokenizes them and converts them to tensorflow datasets
python_dataset = load_and_preprocess_data("python")
js_dataset = load_and_preprocess_data('javascript')

python_dataset_tokenized = preprocess_tokenizeation(python_dataset)
js_dataset_tokenized = preprocess_tokenizeation(js_dataset)

python_Dset_tok_tf = to_tf(python_dataset_tokenized)
js_Dset_tok_tf = to_tf(js_dataset_tokenized)


TypeError: load_and_preprocess_data() got an unexpected keyword argument 'trust_remote_code'